# Optimizing model parameters

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

C:\Users\Luigi\Anaconda3\envs\wekeo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork()

## Hyperparameters

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308364  [    0/60000]
loss: 2.290843  [ 6400/60000]
loss: 2.277952  [12800/60000]
loss: 2.260954  [19200/60000]
loss: 2.237854  [25600/60000]
loss: 2.216485  [32000/60000]
loss: 2.217056  [38400/60000]
loss: 2.187025  [44800/60000]
loss: 2.197622  [51200/60000]
loss: 2.145854  [57600/60000]
Test Error: 
 Accuracy: 53.5%, Avg loss: 2.147697 

Epoch 2
-------------------------------
loss: 2.166515  [    0/60000]
loss: 2.144880  [ 6400/60000]
loss: 2.093831  [12800/60000]
loss: 2.099913  [19200/60000]
loss: 2.035805  [25600/60000]
loss: 1.996863  [32000/60000]
loss: 2.008497  [38400/60000]
loss: 1.931679  [44800/60000]
loss: 1.955929  [51200/60000]
loss: 1.860730  [57600/60000]
Test Error: 
 Accuracy: 59.4%, Avg loss: 1.861116 

Epoch 3
-------------------------------
loss: 1.903061  [    0/60000]
loss: 1.859814  [ 6400/60000]
loss: 1.747060  [12800/60000]
loss: 1.780180  [19200/60000]
loss: 1.669893  [25600/60000]
loss: 1.636827  [32000/600